<a href="https://colab.research.google.com/github/Pilo1961/QuestionAnswer_System/blob/master/code/question_answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sistema de Preguntas y Respuestas
## Métodos Analíticos